In [1]:
from lxml import html
import requests
from string import ascii_lowercase
import sys
import json
from xml.etree.ElementTree import tostring
from IPython.display import clear_output
from time import sleep

# base URL
page_root_url = 'http://www.jokeswarehouse.com/'
# file name for the topics
topics_filename = 'topicsJW.json'
# filename for the jokes
jokes_filename = 'jokesJW.json'
# max joke length
MAX_JOKE_LENGTH = 1000

In [20]:
# request HTML page
page_response = requests.get(page_root_url)
# convert to tree for searching for elements with xpath
tree = html.fromstring(page_response.content)

topics_names = tree.xpath('//td[@bgcolor="#FFFFFF"]//td[@valign="top"]//ul//li//text()')
topics_urls = tree.xpath('//td[@bgcolor="#FFFFFF"]//td[@valign="top"]//ul//li//a/@href')

topics = {}
ix = 0
for topic_name in topics_names:
    if len(topic_name.strip()) > 1:
        topics[topic_name] = topics_urls[ix]
        ix += 1
        
with open(topics_filename, 'w') as fp:
    json.dump(topics, fp)

In [21]:
with open(topics_filename, 'r') as fp:
    topics = json.load(fp)  
topic_names = sorted(topics.keys())
joke_topic_names = [topic_names[0]]

jokes = {}
tot_n_jokes = 0
try:    
    with open(jokes_filename, 'r') as fp:
        jokes = json.load(fp)    
    if len(jokes) == 0:
        print('File does not contain jokes of jokes dict is empty')
    else:
        topics_without_jokes = []
        for topic in topics:
            if topic in jokes.keys():
                n_jokes_topic = 0
                jokes_topic = jokes[topic]
                for joke in jokes_topic:
                    n_jokes_topic+=1
            else:
                topics_without_jokes.append(topic)
            tot_n_jokes += n_jokes_topic
except:
    print('Could not find jokes file: ' + jokes_filename)
    
print('Total number of topics: ' + str(len(topic_names)))
print('Total number of jokes: ' + str(tot_n_jokes))
curr_topic = joke_topic_names[-1]
print('Current topic: ' + curr_topic)
print('Last topic: ' + topic_names[-1])

Could not find jokes file: jokesJW.json
Total number of topics: 28
Total number of jokes: 0
Current topic: Animal Jokes
Last topic: Workplace Jokes


In [ ]:
jokes = {}

for topic_key in topics.keys():
    page_response = requests.get(topics[topic_key], stream=True)
    tree = html.fromstring(page_response.content)
    joke_links = tree.xpath('//td[@bgcolor="#FFFFFF"]//center//a/@href')
    jokes_list = []
    ix = 0
    for joke_link in joke_links:
        ix += 1
        page_response = requests.get(joke_link, stream=True)
        tree = html.fromstring(page_response.content)
        joke_html = tree.xpath('//td[@bgcolor="#FFFFFF"]//p//text()')
        joke_text = ''
        for joke_html_in in joke_html:
            joke_text += joke_html_in.strip()
        formated_joke = joke_text.strip()
        print('Current topic: ' + str(topic_key))
        print('Total number of jokes to process: ' + str(ix) + '/' + str(len(joke_links)))
        print(formated_joke)
        clear_output(wait=True)
        jokes_list.append(formated_joke)
    jokes[curr_topic] = jokes_list
    with open(jokes_filename, 'w') as fp:
        json.dump(jokes, fp)

Current topic: Animal Jokes
Total number of jokes to process: 192/213
I believe that earlier this year the US Postal Service also issued a set of

stamps featuring american racehorses.This must be one of the great ironies of the horse racing industry. If a

horse wins its races it gets put on the front of a postage stamp. If it

loses it gets put on the back!


In [18]:
def get_topic_jokes(topic,topic_page_url,jokes,tot_n_jokes = 0):
    MAX_RANGE_PAGE = 1000
    jokes_list = []   
    ix_joke = 0
    redirected_joke = False
    for ix_page in range(1,MAX_RANGE_PAGE):
        final_page_url = topic_page_url + '/' + str(ix_page)
        page_response = requests.get(final_page_url, stream=True)
        tree = html.fromstring(page_response.content)
        jokes_tree = tree.xpath('//div[@class="joke-text"]')
        if len(jokes_tree) == 0:
            print('Found last joke page')
            break        
        else:            
            for joke in jokes_tree:
                ix_joke += 1
                joke_text_list = [x for x in joke.itertext()] 
                formatted_joke = ' '.join(joke_text_list).strip()
                if formatted_joke == redirect_joke and tot_n_jokes > 0:
                    redirected_joke = True
                    print('Redirected - stopping topic')
                    break
                jokes_list.append(formatted_joke)                           
                tot_n_jokes += 1
                print('Total number of jokes: ' + str(tot_n_jokes))
                print(final_page_url)
                print(topic + ': ' + formatted_joke)
                clear_output(wait=True)
        if redirected_joke:
            break    
    jokes[topic] = jokes_list
    return jokes, tot_n_jokes 

In [ ]:
tot_n_jokes = 0
for topic in topics:  
    jokes, tot_n_jokes = get_topic_jokes(topic,topics[topic],jokes,tot_n_jokes)
    with open(jokes_filename, 'w') as fp:
        json.dump(jokes, fp)
        
with open(jokes_filename, 'r') as fp:
    jokes = json.load(fp)  

Total number of jokes: 3093
http://www.laughfactory.com/jokes/word-play-jokes/28
Word Play Jokes: Knock knock.  Who's there?  Marry.  Marry who?  Marry me, please!
